In [1]:
! python -m pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 103.0 MB/s eta 0:00:00


In [2]:
from gurobipy import Model, GRB, quicksum

# ----------------------
# Dati
# ----------------------
I = ["verdure", "carb", "frutta"]   # risorse (verdure, carboidrati, frutta)
J = ["pro", "fer", "cal"]           # fabbisogni (proteine, ferro, calcio)

# Costo per unità di risorsa i
C = {
    "verdure": 4,
    "carb": 10,
    "frutta": 7,
}

# Fabbisogno richiesto per nutriente j
D = {
    "pro": 20,
    "fer": 30,
    "cal": 10,
}

# Matrice dei coefficienti A[i,j] (quantità di nutriente j per unità di risorsa i)
A = {
    ("verdure", "pro"): 5,   ("carb", "pro"): 15,  ("frutta", "pro"): 4,
    ("verdure", "fer"): 6,   ("carb", "fer"): 10,  ("frutta", "fer"): 5,
    ("verdure", "cal"): 5,   ("carb", "cal"): 3,   ("frutta", "cal"): 12,
}

# Creazione modello
m = Model("dieta")
m.setParam("OutputFlag", 0)

# Creazione variabili decisionali
x = {i: m.addVar(vtype=GRB.INTEGER, lb=0, name=f"x[{i}]") for i in I}

# Definizione funzione obiettivo
m.setObjective(quicksum(C[i] * x[i] for i in I), GRB.MINIMIZE)

# Creazione vincoli
for j in J:
    m.addConstr(quicksum(A[i, j] * x[i] for i in I) >= D[j], name=f"soddisfazione[{j}]")
    m.addConstr(sum(A[i, j] * x[i] for i in I) >= D[j], name=f"soddisfazione[{j}]")

m.update()


# Ottimizza
m.optimize()

# Stampa soluzione
if m.status in (GRB.OPTIMAL, GRB.SUBOPTIMAL):
    print(f"Valore ottimo (costo): {m.objVal:.6g}")
    for i in I:
        print(f"x[{i}] = {x[i].X:.3g}")
else:
    print(f"Ottimizzazione terminata con stato {m.status}")

Restricted license - for non-production use only - expires 2026-11-23
Valore ottimo (costo): 20
x[verdure] = 5
x[carb] = -0
x[frutta] = -0
